In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
class BNB():

    _cVec = None
    _features = None
    _targets = None
    _TS = None
    _M = None
    _indicators = None
    _theta_one = None
    _theta_zero = None
    _theta_j_one = []
    _theta_j_zero = []
    
    def __init__(self,features,targets):
        self._theta_j_one = []
        self._theta_j_zero = []
        self._cVec = CountVectorizer(binary=True ,min_df=0.01) ##TODO ADD min_df, max_df,etc...
        self._targets = targets
        self._TS = targets.shape[0]
        self._numO = (self._targets == 1).sum()
        self._numZ = (self._targets == 0).sum()
        self._theta_one = (self._targets == 1).sum()/targets.shape[0]
        self._theta_zero = (self._targets == 0).sum()/targets.shape[0]
        self._indicators = self._cVec.fit_transform(features)
        self._features = self._cVec.get_feature_names()
        self._M = len(self._features)
        for j in range(self._M):
            self._theta_j_one.append((self._indicators.getcol(j).toarray()[self._targets == 1].sum()+1)/(self._numO+2))
            self._theta_j_zero.append((self._indicators.getcol(j).toarray()[self._targets == 0].sum()+1)/(self._numZ+2))
        
        
    def predict(self,features):
        predL = []
        
        features = self._cVec.transform(features).toarray()
        
        for i in range(features.shape[0]):
            tot = np.log(np.array(self._theta_j_one)/np.array(self._theta_j_zero))
            totmt= (1-np.array(self._theta_j_one))
            totmb=(1-np.array(self._theta_j_zero))
            totm= totmt/totmb
            totm=np.log(totm)
            leftS = features[i]@tot
            RighS = (1-features[i])@totm
            logL = leftS + RighS
            logL = logL + np.log(self._theta_one/self._theta_zero)
            predL.append(logL)
        
        x = np.sign(np.array(predL))
        x[x<0] = 0
        return x
        
    def eval_acc(self,features,targets):
        pred = self.predict(features)
        return np.mean(pred == targets)

df = pd.read_csv('train.csv', encoding='utf-8', index_col=None)
skf = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
X = np.array(df['review'])
Y = np.array(df['sentiment'].replace('positive',1).replace('negative',0))
counter = 0
for index, (train_index, test_index) in enumerate(skf.split(X, Y)):
    print("At k="+str(counter))
    X_train, X_valid, Y_train, Y_valid = X[train_index], X[test_index], Y[train_index], Y[test_index]
    model = BNB(X_train,Y_train)
    print("Accuracy is: ")
    print(model.eval_acc(X_valid,Y_valid))
    del model ##WEIRD?!?!?!??
    counter = counter + 1

At k=0
Accuracy is: 
0.802366272287952
At k=1
Accuracy is: 
0.8080319946675554
At k=2
Accuracy is: 
0.8095
At k=3
Accuracy is: 
0.8089681613602268
At k=4
